<a href="https://colab.research.google.com/github/MKShim/Python/blob/main/SingleView_Lasso1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import linalg as la
from scipy import sparse
import scipy.linalg as linalg
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
mirna = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Opt/dataFiles/all_mirna.csv')
X = np.array(mirna)

In [ ]:
# Simple Lasso + ADMM algorithm

In [ ]:
def coordinate_descent(z, x, nu, rho, lamb):
    e_grad = x + nu*1.0/rho
    rho = 1
    # Regularization term gradient
    # This will have a subgradient, with values as -lambda/rho, lambda/rho OR 0

    # print("prev",z)
    z_t = np.zeros_like(z)

    filter_less = -(1.0*lamb/rho)*(z<0)
    # print("less",filter_less)
    filter_greater = (1.0*lamb/rho)*(z>0)
    # print("gt",filter_greater)

    z_t = e_grad - filter_less - filter_greater
    # print(z_t)
    return(z_t)

In [ ]:
A = X
b = X

#X_t = np.random.randn(7668,7668)
X_t = np.zeros((7668,7668))
z_t = np.zeros((7668,7668))
rho = 1

# nu_t = np.random.randn(d, 1)
nu_t = np.zeros((7668,7668))
num_iterations = 10

print(A.shape,b.shape,X_t.shape,z_t.shape,rho,nu_t.shape)

(2454, 7668) (2454, 7668) (7668, 7668) (7668, 7668) 1 (7668, 7668)


In [ ]:
# Initializations
lamb = 0.1
val = 0.5*np.linalg.norm(A.dot(X_t) - b, ord='fro')**2 + lamb*np.linalg.norm(X_t, ord=1)
print(val)
for iter in range(num_iterations):

    # STEP 1: Calculate X_t
    # This has a closed form solution
    term1 = np.linalg.inv(A.T.dot(A) + rho)
    term2 = A.T.dot(b) + rho*z_t -  nu_t
    X_t = term1.dot(term2)
    # print(term1.shape, term2.shape, X_t.shape)

    # STEP 2: Calculate z_t
    # Taking the prox, we get the lasso problem again, so, using coordinate_descent
    lamb = 0.1
    z_t = coordinate_descent(z_t, X_t, nu_t, rho, lamb)

    # STEP 3: Update nu_t
    nu_t = nu_t + rho*(X_t - z_t)
    val = 0.5*np.linalg.norm(A.dot(X_t) - b, ord='fro')**2 + lamb*np.linalg.norm(X_t, ord=1)
    print(val)
    print(X_t)

3137295108946.448
1.7479263241757466e+29
[[ 3.04132576e+19  2.78660444e+19  2.71730202e+19 ...  2.55775428e+19
   3.14860774e+19  2.99739660e+19]
 [-4.46943145e+18 -4.58540652e+18 -4.09000363e+18 ...  4.34269240e+18
  -4.73347422e+18 -1.75382785e+18]
 [ 2.88190934e+19  2.84891043e+19  2.96849347e+19 ...  3.61998214e+19
   2.98435987e+19  2.81462304e+19]
 ...
 [ 3.80758987e+19  3.60922345e+19  3.76132617e+19 ...  4.12173153e+19
   3.61335886e+19  3.29343156e+19]
 [-8.49838224e+19 -8.18502390e+19 -8.40254719e+19 ... -9.66254536e+19
  -8.39617364e+19 -7.92215527e+19]
 [ 5.81289362e+19  5.65570632e+19  5.64491481e+19 ...  7.60829165e+19
   6.23982409e+19  6.35826383e+19]]
1.041902014498231e+51
[[-7.36833599e+30 -6.92869809e+30 -6.74739479e+30 ... -8.69653769e+30
  -7.92351851e+30 -8.22729137e+30]
 [-3.68489134e+30 -3.56748168e+30 -3.84523862e+30 ... -4.17446355e+30
  -3.36871909e+30 -2.97615776e+30]
 [ 6.68146252e+30  6.36408918e+30  6.35828011e+30 ...  8.14574095e+30
   7.00053820e+30  7.

In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Opt/pyModules')

In [ ]:
from History import *

In [ ]:
# Solves the following problem via ADMM:
# minimize 1/2*|| Ax - b ||_2^2 + \lambda || x ||_1
def lasso(A, b, lamda, rho, alpha):
    QUIET = False
    MAX_ITER = 10
    ABSTOL = 10**(-4)
    RELTOL = 10**(-2)

    m, n = A.shape
    # save a matrix-vector multiply
    Atb=np.dot(A.T,b)
    #Atb = A.T*b

    # ADMM solver
    x = np.zeros([n, 1])
    z = np.zeros([n, 1])
    u = np.zeros([n, 1])

    # cache the factorization
    L, U = factor(A, rho)

    if QUIET is False:
        print('%3s' % 'iter', '%10s' % 'r norm', '%10s' % 'eps pri', '%10s' % 's norm', '%10s' % 'eps dual', '%10s' % 'objective')

    history = History()
    for k in range(0, MAX_ITER):
        # x-update
        q = Atb + rho*(z - u)  # temporary value
        if m >= n:
            x = la.solve(U.todense(), la.solve(L.todense(), q))
        else:
            x = q/rho - np.dot(A.T, la.solve(U.todense(), la.solve(L.todense(), np.dot(A, q))))/rho**2

        # z-update
        zold = z
        x_hat = alpha*x + (1 - alpha)*zold
        z = shrinkage(x_hat + u, lamda/rho)

        # u-update
        u = u + (x_hat - z)

        # diagnostics, reporting, termination checks
        history.addObjval(objective(A, b, lamda, x, z))
        history.addR_norm(la.norm(x - z))
        history.addS_norm(la.norm(-rho*(z - zold)))
        history.addEps_pri(np.sqrt(n)*ABSTOL + RELTOL*np.maximum(la.norm(x), la.norm(-z)))
        history.addEps_dual(np.sqrt(n)*ABSTOL + RELTOL*la.norm(rho*u))

        if QUIET is False:
            print('%3d' % k, '%10.4f' % history.getR_norm()[k], '%10.4f' % history.getEps_pri()[k], '%10.4f' % history.getS_norm()[k],
                  '%10.4f' % history.getEps_dual()[k], '%10.2f' % history.getObjval()[k])

        if history.getR_norm()[k] < history.getEps_pri()[k] and history.getS_norm()[k]<history.getEps_dual()[k]:
            break
    return z, history


def objective(A, b, lamda, x, z):
    a = np.dot(A, x) - b
    p = 1/2*sum((np.multiply(a, a))) + lamda*la.norm(z, 1)
    return p[0]

def shrinkage(x, kappa):
    z = np.maximum(0, (x - kappa)) - np.maximum(0, (-x - kappa))
    #z = np.maximum(0, (x - kappa).getA()) - np.maximum(0, (-x - kappa).getA())
    return z

def factor(A, rho):
    m, n = A.shape
    if m >= n:
        L = la.cholesky(np.dot(A.T, A) + rho*sparse.eye(n))
    else:
        L = la.cholesky(sparse.eye(m) + 1/rho*(np.dot(A, A.T)))

    L = sparse.coo_matrix(L)
    U = sparse.coo_matrix(L.T)

    return L, U

In [ ]:
lambda_max = la.norm(np.dot(X.T, X), np.inf)
lamda = 0.1*lambda_max

In [ ]:
C, history =lasso(X,X,lamda, 1.0, 1.0)

iter     r norm    eps pri     s norm   eps dual  objective
  0    45.5292     0.4640     0.0000     0.4640       0.01
  1    44.2671     0.4514     0.0000     0.9060       0.04
  2    43.3979     0.4427     0.0000     1.3388       0.07
  3    42.7087     0.4358     0.0000     1.7644       0.10
  4    42.1289     0.4300     0.0000     2.1839       0.13
  5    41.6242     0.4250     0.0000     2.5982       0.17
  6    41.1751     0.4205     0.0000     3.0077       0.21
  7    40.7691     0.4164     0.0000     3.4131       0.25
  8    40.3978     0.4127     0.0000     3.8145       0.29
  9    40.0550     0.4093     0.0000     4.2124       0.33
 10    39.7362     0.4061     0.0000     4.6069       0.38
 11    39.4379     0.4031     0.0000     4.9984       0.42
 12    39.1575     0.4003     0.0000     5.3869       0.47
 13    38.8926     0.3977     0.0000     5.7727       0.51
 14    38.6414     0.3952     0.0000     6.1558       0.56
 15    38.4026     0.3928     0.0000     6.5365       0

In [ ]:
print(c)

NameError: ignored

In [ ]:
def sparse_coding( Y, D, X, rho, num_iterations, Z=None, U=None ):
    if Z is None:
        Z = X.copy()
    if U is None:
        U = X - Z

    # precompute solve and part of RHS
    iDtD = numpy.linalg.inv( numpy.dot(D.transpose(),D) + numpy.eye(D.shape[1]) )
    DtY  = numpy.dot( D.transpose(), Y )

    for iter in range(num_iterations):
        print('    Sparse coding iteration [{}/{}]'.format(iter+1,num_iterations) )
        # primary update
        X = numpy.dot( iDtD, DtY + Z - U )

        # splitting variable update
        T = X + U
        Z = numpy.maximum( numpy.abs(T) - rho, 0.0)*numpy.sign(T)

        # lagrange multiplier update
        U = T - Z
    return X, Z, U

In [ ]:
sparse_coding(X,X,C)

NameError: ignored

In [ ]:
#hyper-parameters in the ADMM
mu1=1e-6
mu2 = 1e-5
mu3 = 4e-5
tau = 0.0001

# Number of iterations
iters = 5

In [ ]:
# Model parameter
X=tf.Variable(mirna)

In [ ]:
#loss
loss=tf.reduce_sum(tf.square(linear_model-X))

<tf.Variable 'Variable:0' shape=(2454, 7668) dtype=float64, numpy=
array([[0.45480001, 0.37029999, 0.48710001, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.16949999],
       [0.        , 0.        , 0.        , ..., 0.        , 0.30610001,
        0.16949999],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])>

In [ ]:
# opt
optimizer=tf.

In [ ]:
## dual decomposition ~ parallel coding CHK

In [ ]:
C=linalg.solve(X,X)

ValueError: ignored

In [ ]:
C,residue,r,s=linalg.lstsq(X,X)